In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from datetime import datetime
from pprint import pprint
import configparser
import ezomero
from ezomero import rois
# https://downloads.openmicroscopy.org/omero/5.4.1/api/omero/gateway/model/ImageData.html

In [2]:
home = str(Path.home())
config = configparser.RawConfigParser()
config.read(r'{}\Documents\myKeyVariables.cfg'.format(home))   
envVars = dict(config.items('OMERO'))

# Settings for OMERO
user = envVars['user']
password = envVars['pass']
host = envVars['url']
port = 4064 #4080
secure = True #False
outDir = envVars['pointsdownloaded']

In [3]:
conn = ezomero.connect(user, password, 'Wang.C_Lab', host=host, port=port, secure=secure)
#ezomero.print_projects(conn) 
allPrjts = ezomero.get_project_ids(conn, across_groups=False)
print("There are {} projects to investigate.".format(len(allPrjts)))

allRecords = []

for pj in allPrjts:
    allSamps = ezomero.get_dataset_ids(conn, project=pj)
    print("  There are {} datasets to investigate.".format(len(allSamps)))
    
    for sm in allSamps:
        imgs = ezomero.get_image_ids(conn, dataset=sm)
        for im in imgs:
            imgDesc, pix = ezomero.get_image(conn, im, xyzct=False, no_pixels=True)
            aquiredDate = ''
            if imgDesc.getAcquisitionDate() is not None:
                aquiredDate = imgDesc.getAcquisitionDate().strftime("%Y-%m-%d %H:%M:%S")
            rw = [pj, sm, im, imgDesc.getName(), imgDesc.getInstrument(), imgDesc.getDescription(), 
                 aquiredDate, imgDesc.getSizeX(), imgDesc.getSizeY(),
                 imgDesc.getSizeZ(), imgDesc.getSizeC(), imgDesc.getSizeT(), ezomero.get_pyramid_levels(conn, im)]
            allRecords.append(rw)

There are 5 projects to investigate.
  There are 3 datasets to investigate.
  There are 6 datasets to investigate.
  There are 10 datasets to investigate.
  There are 4 datasets to investigate.
  There are 1 datasets to investigate.


In [4]:
fullData = pd.DataFrame(allRecords)
fullData.columns = ['Project_id', 'Dataset_id', 'Image_id', 'Image_Name', 'Instrument', 'Image_Description', 
                    'Upload_Date','X','Y','Z','C','T','Pyramidal_Levels']

fullData.to_csv(r'{}\Documents\DIMILab_OmeroImageSizes.tsv'.format(home), sep='\t')